In [1]:
import matplotlib.widgets
import ipywidgets as widgets
import pandas as pd
from ipywidgets import interact, interactive, interactive_output, fixed, FloatSlider, Layout
import matplotlib.pyplot as plt
import numpy as np
import math
import sys

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [2]:
def generate_sample(n, low, high):
    # arr = np.random.uniform(low=low, high=high, size=n).round(decimals=2)
    arr = np.random.randint(low=low, high=high, size=n)
    return arr

In [3]:
class DiscreteSamplingDistribution:
    def __init__(self, sample, n):
        (unique, counts) = np.unique(sample, return_counts=True)
        self.sample = sample
        self.n = n
        self.df = pd.DataFrame({'ni': counts, "wi": counts / n}, unique)
        self.size = self.df.shape[0]

    def show_database(self):
        pd.set_option('display.max_rows', self.df.shape[0]+1)
        styler = self.df.style.set_properties(**{'background-color': '#B0C4DE', 'color': '#154360'})
        display(styler) 

    @staticmethod
    def draw_plot(x_label, y_label, title):
        fig, ax = plt.subplots(figsize=(10, 7))
        ax.set_title(title, color='black', size=25, fontweight='bold', pad=30)
        ax.set_ylabel(y_label, size=15, fontweight='bold')
        ax.set_xlabel(x_label, size=15, fontweight='bold')
        ax.patch.set_facecolor('#B0C4DE')
        fig.patch.set_facecolor('#154360')
        plt.grid(zorder=0)

    def counts_diagram(self):
        DiscreteSamplingDistribution.draw_plot("x", "n", "Діаграма Частот")
        plt.bar(self.df.index, self.df['ni'], color="#CD5C5C", zorder=3, edgecolor='red')
        plt.legend(['Частота'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(self.df['ni'])
        plt.show()

    def frequency_diagram(self):
        DiscreteSamplingDistribution.draw_plot("x", "n", "Діаграма Відносних Частот")
        plt.bar(self.df.index, self.df['wi'], color="#CD5C5C", zorder=3, edgecolor='red')
        plt.legend(['Відносна частота'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(self.df['wi'])
        plt.show()
    
    def counts_polygon(self):
        DiscreteSamplingDistribution.draw_plot("x", "n", "Полігон Частот")
        plt.plot(self.df.index, self.df['ni'], '-ok', color="red")
        plt.legend(['Частота'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(self.df['ni'])
        plt.show()

    def frequency_polygon(self):
        DiscreteSamplingDistribution.draw_plot("x", "n", "Полігон Відносних Частот")
        plt.plot(self.df.index, self.df['wi'], '-ok', color="red")
        plt.legend(['Відносна частота'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(self.df['wi'])
        plt.show()

    def count_ecdf_func(self):
        ecdf = np.copy(self.df.wi.to_numpy())
        for i in range(1, self.size):
            ecdf[i] += ecdf[i-1]
        return ecdf

    def print_ecdf_func(self, ecdf):
        res = f"0\t x < {self.df.index[0]};\n"
        for i, p in enumerate(ecdf[:-1]):
            res += f"{round(p, 2)}\t {self.df.index[i]} <= x < {self.df.index[i + 1]};\n"
        res += f"{round(ecdf[self.size-1], 2)}\t x >= {round(self.df.index[self.size-1], 2)};\n"
        print(res)

    def drow_ecdf(self, ecdf):
        xx = np.empty([self.df.index.size + 1, 2])
        xx[0] = np.array([self.df.index[0]-1, self.df.index[0]])
        for i in range(1, self.df.index.size):
            xx[i] = np.array([self.df.index[i - 1], self.df.index[i]])
        xx[self.df.index.size] = np.array([self.df.index[self.df.index.size - 1], self.df.index[self.df.index.size - 1] + 1])
        ecdf_new = np.insert(np.copy(ecdf), 0, 0.0)
 
        DiscreteSamplingDistribution.draw_plot("x", "F(x)", "Емпірична Функція Розподілу")
        plt.hlines(ecdf_new, xx[:, 0], xx[:, 1], color = 'red')
        plt.vlines(self.df.index, 0, ecdf, color = 'red', ls = "--", linewidth=1)
        plt.legend(['Емпірична функція'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(ecdf)
        plt.show()

    # Вибіркове середнє
    def get_mean(self):
        return np.sum(self.sample) / self.n

    # Мода
    def get_mode(self): 
        return self.df.index[np.argwhere(self.df.ni.to_numpy() == np.amax(self.df.ni))].T[0]

    # Медіана
    def get_median(self):
        return (self.sample[math.ceil(float(self.n)/2 + 0.5) - 1] + self.sample[math.floor(float(self.n)/2) - 1]) / 2

    # Розмах вибірки
    def get_range(self):
        return np.max(self.sample) - np.min(self.sample)

    # Девіація
    def get_deviation(self):
        mean = self.get_mean()
        norm = self.df.index - mean
        norm_square = np.square(norm)* self.df.ni
        return np.sum(norm_square)

    # Варіанса
    def get_variance(self):
        return self.get_deviation() / (self.n - 1)

    # Стандарт
    def get_standard_deviation(self):
        return np.sqrt(self.get_variance())

    # Варіація
    def get_variation(self):
        return self.get_standard_deviation() / self.get_mean()

    # Вибіркова дисперсія
    def get_dispersion(self):
        return self.get_deviation() / self.n

    # Вибіркове середнє квадратичне відхилення
    def get_standard_error(self):
        return np.sqrt(self.get_dispersion())

    def get_quantiles(self):
        data = {(4, "Квартил"): [], (8, "Октил"): [], (10, "Децил"): [],
                (100, "Центил"): [], (1000, "Міліл"): []}

        for k in data.keys():
            if len(self.sample) % k[0] == 0:
                data[k] = [self.sample[int(len(self.sample) / k[0] * (i + 1))] for i in range(k[0] - 1)]

        for k, v in data.items():
            if v:
                print(f"\t{k[1]}і: {v}")
                print(f"\tІнтер{k[1].lower()}ьна широта: {v[len(v) - 1] - v[0]}\n")

    # Початковий момент
    def get_initial_moment(self, order):
        return np.sum(np.power(self.df.index * self.df.ni, order))/self.n

    # Центральний момент
    def get_central_moment(self, order):
        return np.sum(np.power((self.df.index - self.get_mean()), order)* self.df.ni) / self.n

    # Асиметрія
    def get_skewness(self):
        skewness = self.get_central_moment(3)/pow(self.get_central_moment(2), 3.0/2)
        print(f"* Асиметрія: {skewness} => ", end="")
        if skewness > 0: print("Статистичний матеріал скошений вправо.")
        elif skewness < 0: print("Статистичний матеріал скошений вліво.")
        else:  print("Статистичний матеріал симетричний.")

    # Ексцес
    def get_kurtosis(self):
        kurtosis = self.get_central_moment(4) / pow(self.get_central_moment(2), 2) - 3
        print(f"* Ексцес: {kurtosis} => ", end="")
        if kurtosis > 0: print("Статистичний матеріал високовершинний.")
        elif kurtosis < 0: print("Статистичний матеріал низьковершинний.")
        else:  print("Статистичний матеріал нормальновершинний.")

In [4]:
class IntervalSamplingDistribution(DiscreteSamplingDistribution):
    def __init__(self, sample, n, k):
        super().__init__(sample, n)
        self.k = k
        delta = (self.df.index[self.size-1] - self.df.index[0]) / k
        interval_x = np.empty([k, 2])
        am = self.df.index[0]
        for i in range(k):
            typle = np.array([am, am + delta])
            interval_x[i] = typle
            am += delta

 
        interval_counts = np.zeros(self.k).astype(int)
        interval_counts[0] = self.df.ni.to_numpy()[0]

        for i, (i1, i2) in enumerate(interval_x):
            interval_counts[i] += np.sum(self.df.ni[(self.df.index > i1) & (self.df.index <= i2)])

        self.interval_df = pd.DataFrame({'ni': interval_counts,
                                         "wi": interval_counts / self.n,
                                         "x0": interval_x[:, 0],
                                         "x1": interval_x[:, 1]},
                                        IntervalSamplingDistribution.interval_x_toString(interval_x))
        self.interval_df["middle_intervals"] = (self.interval_df.x0.to_numpy() + self.interval_df.x1.to_numpy()) / 2

    # Вибіркове середнє
    def get_mean_interval(self):
        return np.sum(self.interval_df.middle_intervals * self.interval_df.ni) / self.n

    # Мода
    def get_mode_interval(self):
        inds = np.argwhere(self.interval_df.ni.to_numpy() == np.amax(self.interval_df.ni)).T
        res = []
        for ind in inds[0]:
            z_i_1, z_i = self.interval_df.x0[ind], self.interval_df.x1[ind]
            n_i_1 = 0 if ind == 0 else self.interval_df.ni[ind - 1]
            n_i1 = 0 if ind == self.k - 1 else self.interval_df.ni[ind + 1]
            n_i = self.interval_df.ni[ind]
            res.append(z_i_1 + ((n_i - n_i_1) * (z_i - z_i_1)) / ((n_i - n_i_1) + (n_i - n_i1)))
        return res

    # Медіана
    def get_median_interval(self):
        val = self.n / 2
        i, m = 0, 0
        while m < val:
            m += self.interval_df.ni[i]
            i += 1
        z_i_1, z_i = self.interval_df.x0[i - 1], self.interval_df.x1[i - 1]
        return z_i_1 + ((z_i - z_i_1) * (val - m + self.interval_df.ni[i - 1])) / self.interval_df.ni[i - 1]

    # Розмах вибірки
    def get_range_interval(self):
        return self.interval_df.x1[self.k - 1]  - self.interval_df.x0[0] 

    # Девіація
    def get_deviation_interval(self):
        mean = self.get_mean_interval()
        norm = self.interval_df.middle_intervals - mean
        norm_square = np.square(norm) * self.interval_df.ni
        return np.sum(norm_square)

    # Варіанса
    def get_variance_interval(self):
        return self.get_deviation_interval() / (self.n - 1)

    # Стандарт
    def get_standard_deviation_interval(self):
        return np.sqrt(self.get_variance_interval())

    # Варіація
    def get_variation_interval(self):
        return self.get_standard_deviation_interval() / self.get_mean_interval()

    # Вибіркова дисперсія
    def get_dispersion_interval(self):
        return self.get_deviation_interval() / self.n

    # Вибіркове середнє квадратичне відхилення
    def get_standard_error_interval(self):
        return np.sqrt(self.get_dispersion_interval())

    # Початковий момент
    def get_initial_moment_interval(self, order):
        return np.sum(np.power(self.interval_df.middle_intervals * self.interval_df.ni, order))/self.n

    # Центральний момент
    def get_central_moment_interval(self, order):
        return np.sum(np.power((self.interval_df.middle_intervals - self.get_mean()), order)* self.interval_df.ni) / self.n

    # Асиметрія
    def get_skewness_interval(self):
        skewness = self.get_central_moment_interval(3)/pow(self.get_central_moment_interval(2), 3.0/2)
        print(f"* Асиметрія: {skewness} => ", end="")
        if skewness > 0: print("Статистичний матеріал скошений вправо.")
        elif skewness < 0: print("Статистичний матеріал скошений вліво.")
        else:  print("Статистичний матеріал симетричний.")

    # Ексцес
    def get_kurtosis_interval(self):
        kurtosis = self.get_central_moment_interval(4) / pow(self.get_central_moment_interval(2), 2) - 3
        print(f"* Ексцес: {kurtosis} => ", end="")
        if kurtosis > 0: print("Статистичний матеріал високовершинний.")
        elif kurtosis < 0: print("Статистичний матеріал низьковершинний.")
        else:  print("Статистичний матеріал нормальновершинний.")


    def show_integral_database(self):
        styler = self.interval_df[["ni", "wi"]].style.set_properties(**{'background-color': '#B0C4DE', 'color': '#154360'})
        display(styler) 
        
    def draw_histogram(self):
        h = self.interval_df.ni/(self.interval_df.x1-self.interval_df.x0)
        DiscreteSamplingDistribution.draw_plot("x", "h", "Гістограма Частот")
        k = np.append(np.array([self.interval_df.x0[0], self.interval_df.x1[0]]), self.interval_df.x1[1:])
        s = k + 0.0001

        plt.hist(s[:-1], bins=k, weights=h, color='#5499C7', edgecolor='#154360', zorder=3)
        plt.xticks(k)
        plt.yticks(h)
        plt.show()

    def draw_histogram_frequency(self):
        h = self.interval_df.wi/(self.interval_df.x1-self.interval_df.x0)
        DiscreteSamplingDistribution.draw_plot("x", "h", "Гістограма Відносних Частот")
        k = np.append(np.array([self.interval_df.x0[0], self.interval_df.x1[0]]), self.interval_df.x1[1:])
        s = k + 0.0001

        plt.hist(s[:-1], bins=k, weights=h, color='#5499C7', edgecolor='#154360', zorder=3)
        plt.xticks(k)
        plt.yticks(h)
        plt.show()
    
    @staticmethod
    def interval_x_toString(interval_x):
        res = np.array([f"({round(x[0], 2)}, {round(x[1], 2)}]" for x in interval_x])
        res[0] = f"[{round(interval_x[0][0], 2)}, {round(interval_x[0][1], 2)}]"
        return res

In [5]:
def printb(string):
    print('\033[1m' + string,  end=" ")
    
def discrete_stats(dist):
    printb("* Вибіркове середнє: ")
    print(dist.get_mean())
    printb("* Мода: ")
    print(dist.get_mode())
    printb("* Медіана: ")
    print(dist.get_median())
    printb("* Розмах вибірки: ")
    print(dist.get_range())
    printb("* Девіація: ")
    print(dist.get_deviation())
    printb("* Варіанса: ")
    print(dist.get_variance())
    printb("* Стандарт: ")
    print(dist.get_standard_deviation())
    printb("* Варіація: ")
    print(dist.get_variation())
    printb("* Вибіркова дисперсія: ")
    print(dist.get_dispersion())
    printb("* Вибіркове середнє квадратичне відхилення: ")
    print(dist.get_standard_error())
    printb("* Інтерквантильні широти: \n")
    dist.get_quantiles()
    dist.get_skewness()
    dist.get_kurtosis()

def interval_stats(dist):
    printb("* Вибіркове середнє: ")
    print(dist.get_mean_interval())
    printb("* Мода: ")
    print(dist.get_mode_interval())
    printb("* Медіана: ")
    print(dist.get_median_interval())
    printb("* Розмах вибірки: ")
    print(dist.get_range_interval())
    printb("* Девіація: ")
    print(dist.get_deviation_interval())
    printb("* Варіанса: ")
    print(dist.get_variance_interval())
    printb("* Стандарт: ")
    print(dist.get_standard_deviation_interval())
    printb("* Варіація: ")
    print(dist.get_variation_interval())
    printb("* Вибіркова дисперсія: ")
    print(dist.get_dispersion_interval())
    printb("* Вибіркове середнє квадратичне відхилення: ")
    print(dist.get_standard_error_interval())
    dist.get_skewness_interval()
    dist.get_kurtosis_interval()


In [6]:
n_widget = widgets.BoundedIntText(value=50, step=1, description='Обсяг вибірки:', layout=Layout(width='20%', height='30px'), style={'description_width': 'initial'})
range_min = widgets.BoundedFloatText(value=0, step=1, description='Початок: ', layout=Layout(width='20%', height='30px'))
range_max = widgets.BoundedFloatText(value=20, step=1, description='Кінець: ', layout=Layout(width='20%', height='30px'))
t1 = widgets.ToggleButton(description='Згенерувати вибірку', button_style='info', icon='check', layout=Layout(width='20%', height='30px'))
t2 = widgets.ToggleButton(value=False, description='Дискретний розподіл', button_style='info', icon='check', layout=Layout(width='20%', height='30px'))
t3 = widgets.ToggleButton(description='Інтервальний розподіл', button_style='info', icon='check', layout=Layout(width='20%', height='30px'))
out1 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))
out2 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))
out3 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))

choice = widgets.ToggleButtons(options=['Рандом', 'Власна'], value=None, description='Вибірка:', button_style='info')
text = widgets.Textarea(value='1 2 3 4 5', description='Вибірка:' , layout=Layout(width='20%', height='30px'))


In [7]:
def part1():
    @interact(choice = choice)
    def f1(choice):
        global sample
        global n

        if (choice == 'Рандом'):
            @interact(Number=n_widget, Start=range_min, End=range_max)
            def function1(Number, Start, End):
                global sample
                global n

                n = Number
                sample = generate_sample(n, Start, End)
                
        elif (choice == 'Власна'):
            @interact(text=text)
            def function2(text):
                global sample
                global n
                
                sample = np.array([float(i) for i in text.split(" ")])
                n = sample.size

In [8]:
def part2():
    @interact(toggle1=t1)
    def function3(toggle1):
        with out1:
            global sample
            global n

            if toggle1==True:            
                print(" Вибірка: ", sample)
                sample = np.sort(sample)
                print("\n\n Варіаційний ряд: ", sample)    
            else:
                out1.clear_output()

In [9]:
def part3():
    @interact(toggle2=t2)
    def function4(toggle2):
        with out2:
            if toggle2==True:  
                global sample
                global n
                
                dist = DiscreteSamplingDistribution(sample, n)
                
                print("\n\n Частотна таблиця: ")
                dist.show_database()

                @interact()
                def disc(counts_diagram=False, frequency_diagram=False, counts_polygon=False, frequency_polygon=False, ecdf=False, stats=False):
                    if counts_diagram:
                        dist.counts_diagram()
                    if frequency_diagram:
                        dist.frequency_diagram()
                    if counts_polygon:
                        dist.counts_polygon()
                    if frequency_polygon:
                        dist.frequency_polygon()
                    if ecdf:
                        ecdf_val = dist.count_ecdf_func()
                        dist.print_ecdf_func(ecdf_val)
                        dist.drow_ecdf(ecdf_val)
                    if stats:
                        discrete_stats(dist)
            else:
                out2.clear_output()

In [10]:
def part4():
    @interact(toggle3=t3)
    def function5(toggle3):
        with out3:
            if toggle3==True:  
                global sample
                global n

                k = widgets.IntSlider(value=int(np.log2(n)) + 1, min=1, max=n, step=1, description='Кількість Інтервалів: ', layout=Layout(width='40%', height='30px'), style={'description_width': 'initial'})

                @interact(k=k)
                def div(k, counts_histogram=False, frequency_histogram = False, stats=False):
                    interval_dist = IntervalSamplingDistribution(sample, n, k)
                    interval_dist.show_integral_database()
                    if counts_histogram:
                        interval_dist.draw_histogram()
                    if frequency_histogram:
                        interval_dist.draw_histogram_frequency()
                    if stats:
                        interval_stats(interval_dist)
            else:
                out3.clear_output() 

In [11]:
part1()
part2()
part3()
part4()

display(out1) 
display(out2) 
display(out3) 
# 8 12 6 1 14 6 1 16 8 8 10 11 5 8 11 16 2 10 18 18 3 10 6 12 17 17 3 15 19 13 15 19 9 2 8 16 12 16 16 14 14 17 13 12 14 12 13 8 5 19

interactive(children=(ToggleButtons(button_style='info', description='Вибірка:', options=('Рандом', 'Власна'),…

interactive(children=(ToggleButton(value=False, button_style='info', description='Згенерувати вибірку', icon='…

interactive(children=(ToggleButton(value=False, button_style='info', description='Дискретний розподіл', icon='…

interactive(children=(ToggleButton(value=False, button_style='info', description='Інтервальний розподіл', icon…

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))